In [1]:
from ctypes import *
from time import sleep

In [2]:
# Custom Exceptions
class Error(Exception):
    """Base class for exceptions in this module.
    :param msg: Error message associated with the exception
    :type msg: str
    :ivar msg: Error message associated with the exception
    """

    def __init__(self, msg):
        self.msg = msg

    def __str__(self):
        return self.msg


class DLLError(Error):
    """Exception raised opening dgilib.dll. dgilib.dll could not be found in the specicied path"""

    pass


class DeviceIndexError(Error):
    """Exception raised selecting device: the device_index exeeds the device_count"""

    pass


class DeviceReturnError(Error):
    """Exception raised: DGILib returned non-zero value"""

    pass


class DeviceConnectionError(Error):
    """Exception raised: Could not connect to the device"""

    pass

In [3]:
GET_STRING_SIZE = 100
NUM_INTERFACES = 10
NUM_CONFIG_IDS = 10
NUM_CALIBRATION = 255
BUFFER_SIZE = 10000000

INTERFACE_TIMESTAMP  = 0x00 # Service interface which appends timestamps to all received events on associated interfaces.
INTERFACE_SPI        = 0x20 # Communicates directly over SPI in Slave mode.
INTERFACE_USART      = 0x21 # Communicates directly over USART in Slave mode.
INTERFACE_I2C        = 0x22 # Communicates directly over I2C in Slave mode.
INTERFACE_GPIO       = 0x30 # Monitors and controls the state of GPIO pins.
INTERFACE_POWER_DATA = 0x40 # Receives data from the attached power measurement co-processors.
INTERFACE_POWER_SYNC = 0x41 # Receives sync events from the attached power measurement co-processors.
INTERFACE_RESERVED   = 0xFF # Special identifier used to indicate no interface.

In [20]:
# https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER

"""This module provides Python bindings for DGILib."""

__license__ = "MIT"
__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "1.6.0"
__revision__ = " $Id: digilib.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"


class DGILib(object):
    """Python bindings for DGILib.
    
    DGILib is a Dynamic-Link Library (DLL) to help software applications communicate with Data Gateway
    Interface (DGI) devices. See the Data Gateway Interface user guide for further details. DGILib handles
    the low-level USB communication and adds a level of buffering for minimizing the chance of overflows.
    The library helps parse data streams of high complexity. The timestamp interface is parsed and split into
    separate buffers for each data source. The power interface is optionally parsed and calibrated using an
    auxiliary API.
    
    :Example:

    >>> with DGILib() as dgilib:
        dgilib.info(True)
    major_version: 5, minor_version: 7, build_number: 244
    """

    def __init__(
        self, dgilib_path="dgilib", device_index=None, device_sn=None, verbose=1
    ):
        """
        :param dgilib_path: Path to dgilib.dll (More info at: https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER)
        :type dgilib_path: str
        :param device_index: index of the device to use, only usefull if multiple devices are connected
            (default is None, will resolve to 0)
        :type device_index: int or None
        :param device_sn: the serial number of the device to use. Has higher priority than device_index
            (default is None, will resolve to serial number of device 0)
        :type device_sn: str or None
        :param verbose: set to a positive number to print more status messages (default is 0)
        :type verbose: int
        :raises: :exc:`DLLError`
        """

        # Load the dgilib.dll
        try:
            self.dgilib = cdll.LoadLibrary(dgilib_path)
        except OSError as e:
            raise DLLError(
                f"dgilib.dll could not be found in the specicied path: {dgilib_path}."
            )

        self.device_index = device_index
        self.device_sn = device_sn
        self.verbose = verbose

    def info(self, print_info=False):
        """Get the build information of DGILib.
        
        :param print_info: A flag used to print the build information to the console (default is False)
        :type print_info: bool
        :return:  Version information of DGILib:
            - major_version: the major_version of DGILib
            - minor_version: the minor_version of DGILib
            - build_number: the build number of DGILib. 0 if not supported
        :rtype: tuple
        """

        major_version = self.dgilib.get_major_version()
        minor_version = self.dgilib.get_minor_version()
        build_number = self.dgilib.get_build_number()

        if self.verbose or print_info:
            print(
                f"major_version: {major_version}\nminor_version: {minor_version}\nbuild_number: {build_number}"
            )

        return (major_version, minor_version, build_number)

    def __enter__(self):
        """
        :raises: :exc:`DeviceIndexError`
        :raises: :exc:`DeviceConnectionError`
        """

        self.discover()
        device_count = self.get_device_count()

        if self.device_sn is None:
            if self.device_index is None:
                self.device_index = 0
            elif self.device_index > device_count - 1:
                raise DeviceIndexError(
                    f"Discovered {device_count} devices so could not select device with index {self.device_index}."
                )
            self.device_sn = self.get_device_serial(self.device_index)

        # UNTESTED:
        # if msd_mode:
        #     res = self.set_mode(sn, 1)
        #     print(f"\t{res} set_mode 1")

        self.connect()
        c_status = self.connection_status()
        if c_status:
            raise DeviceConnectionError(
                f"Could not connect to device. Connection status: {c_status}."
            )

        self.interface_list()
        self.interface_enable(None, INTERFACE_GPIO, True)

        self.start_polling()

        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.interface_disable(None, INTERFACE_GPIO)
        self.stop_polling()
        self.disconnect()
        print("bye")

    def discover(self):
        """`discover`

        Triggers a scan to find available devices in the system. The result will be immediately available through
        the `get_device_count`, `get_device_name` and `get_device_serial` functions.

        `void discover(void)`
        """

        self.dgilib.discover()

    def get_device_count(self):
        """`get_device_count`

        Returns the number of devices detected.

        `int get_device_count(void)`
        
        :return: The number of devices detected
        :rtype: int
        """

        device_count = self.dgilib.get_device_count()
        if self.verbose:
            print(f"device_count: {device_count}")
        return device_count

    def get_device_name(self, index=None):
        """`get_device_name`

        Gets the name of a detected device.

        `int get_device_name(int index, char* name)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *index* | Index of device ranges from 0 to `get_device_count` - 1 |
        | *name* | Pointer to buffer where name of device can be stored. 100 or more bytes must be allocated |
        +------------+------------+
        
        :param index: Index of device ranges from 0 to `get_device_count` - 1
            (defaults to self.device_index)
        :type index: int or None
        :raises: :exc:`DeviceReturnError`
        :return: The name of a detected device
        :rtype: str
        """

        if index is None:
            index = self.device_index

        name = create_string_buffer(GET_STRING_SIZE)
        res = self.dgilib.get_device_name(index, byref(name))
        if self.verbose:
            print(f"\t{res} get_device_name: {name.value}")
        if res:
            raise DeviceReturnError(f"get_device_name returned: {res}")
        return name.value

    def get_device_serial(self, index=None):
        """`get_device_serial`

        Gets the serial number of a detected device.

        `int get_device_serial(int index, char* sn)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *index* | Index of device ranges from 0 to `get_device_count` - 1 |
        | *sn* | Pointer to buffer where the serial number of the device can be stored. 100 or more bytes must be allocated. This is used when connecting to a device |
        +------------+------------+
        
        :param index: Index of device ranges from 0 to `get_device_count` - 1
            (defaults to self.device_index)
        :type index: int or None
        :raises: :exc:`DeviceReturnError`
        :return: The serial number of a detected device
        :rtype: str
        """

        if index is None:
            index = self.device_index

        sn = create_string_buffer(GET_STRING_SIZE)
        res = self.dgilib.get_device_serial(index, byref(sn))
        if self.verbose:
            print(f"\t{res} get_device_serial: {sn.value}")
        if res:
            raise DeviceReturnError(f"get_device_serial returned: {res}")
        return sn.value

    def is_msd_mode(self, sn=None):
        """`is_msd_mode`
        EDBG devices can be set to a mass storage mode where the DGI is unavailable. In such cases the 
        device is still detected by DGILib, but it won't be possible to directly connect to it. This command is used 
        to check if the device is in such a mode.

        A non-zero return value indicates that the mode must be changed by `set_mode` before proceeding.

        `int is_msd_mode(char* sn)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *sn* | Serial number of the device to check |
        +------------+------------+
        
        :param sn: Serial number of the device to check (defaults to self.device_sn)
        :type sn: str or None
        :return: A non-zero return value indicates that the mode must be changed by `set_mode` before proceeding.
        :rtype: int
        """

        if sn is None:
            sn = self.device_sn

        msd_mode = self.dgilib.is_msd_mode(sn)
        if self.verbose:
            print(f"msd_mode: {msd_mode}")
        return msd_mode

    def set_mode(self, sn=None, nmbed=1):
        """`set_mode`
        This function is used to temporarily set the EDBG to a specified mode.

        `int set_mode(char* sn, int nmbed)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *sn* | Serial number of the device to set |
        | *nmbed* | 0 - Set to mbed mode. 1 - Set to DGI mode |
        +------------+------------+
        
        :param sn: Serial number of the device to set (defaults to self.device_sn)
        :type sn: str or None
        :param nmbed: 0 - Set to mbed mode. 1 - Set to DGI mode (defaults to DGI mode)
        :type nmbed: int
        :raises: :exc:`DeviceReturnError`
        """

        if sn is None:
            sn = self.device_sn

        res = self.dgilib.set_mode(sn, nmbed)
        if self.verbose:
            print(f"\t{res} set_mode {nmbed}")
        if res:
            raise DeviceReturnError(f"get_device_serial returned: {res}")

    def connect(self, sn=None, dgi_hndl=None):
        """`connect`
        Opens a connection to the specified device. This function must be called prior to any function requiring
        the connection handle.

        `int connect(char* sn, uint32_t* dgi_hndl_p)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *sn* | Buffer holding the serial number of the device to open a connection to |
        | *dgi_hndl_p* | Pointer to a variable that will hold the handle of the connection |
        +------------+------------+
        
        :param sn: Serial number of the device (defaults to self.device_sn)
        :type sn: str or None
        :param dgi_hndl: Variable that will hold the handle of the connection (defaults to c_uint())
        :type dgi_hndl: c_uint() or None
        :raises: :exc:`DeviceReturnError`
        """

        if sn is None:
            sn = self.device_sn
        if dgi_hndl is None:
            dgi_hndl = c_uint()

        self.dgi_hndl = dgi_hndl

        # Initialize (not in manual, exists in dgilib.h)
        # self.dgilib.Initialize(byref(dgi_hndl))

        res = self.dgilib.connect(sn, byref(dgi_hndl))
        if self.verbose:
            print(f"\t{res} connect")
        if res:
            raise DeviceReturnError(f"connect returned: {res}")

    def disconnect(self, dgi_hndl=None):
        """`disconnect`
        Closes the specified connection.

        `int disconnect(uint32_t dgi_hndl)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :raises: :exc:`DeviceReturnError`
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl

        res = self.dgilib.disconnect(dgi_hndl)
        if self.verbose:
            print(f"\t{res} disconnect")
        if res:
            raise DeviceReturnError(f"disconnect returned: {res}")

        # UnInitialize (not in manual, exists in dgilib.h)
        # self.dgilib.UnInitialize(dgi_hndl)

    def connection_status(self, dgi_hndl=None):
        """`connection_status`
        Verifies that the specified connection is still open.

        `int connection_status(uint32_t* dgi_hndl)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :return: A non-zero return value indicates a connection error.
        :rtype: int
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl

        c_status = self.dgilib.connection_status(dgi_hndl)
        if self.verbose:
            print(f"connection_status: {c_status}")

        return c_status

    def get_fw_version(self, dgi_hndl=None, print_info=False):
        """`get_fw_version`
        Gets the firmware version of the DGI device connected. Note that this is the version of the DGI device,
        and not the tool.
        
        `int get_fw_version(uint32_t dgi_hndl, unsigned char* major, unsigned char* minor)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *major* | Pointer to a variable where the major version will be stored |
        | *minor* | Pointer to a variable where the minor version will be stored |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :param print_info: A flag used to print the firmware information to the console (default is False)
        :type print_info: bool
        :return: Version information of the DGI device:
            - major_fw: the major firmware version of the DGI device connected
            - minor_fw: the minor firmware version of the DGI device connected
        :rtype: tuple(int, int)
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl

        major_fw = c_ubyte()
        minor_fw = c_ubyte()
        res = self.dgilib.get_fw_version(dgi_hndl, byref(major_fw), byref(minor_fw))
        if self.verbose:
            print(f"\t{res} get_fw_version")
        if res:
            raise DeviceReturnError(f"get_fw_version returned: {res}")
        if self.verbose or print_info:
            print(f"major_fw: {major_fw.value}\nminor_fw: {minor_fw.value}")

        return (major_fw.value, minor_fw.value)

    def start_polling(self, dgi_hndl=None):
        """`start_polling`
        This function will start the polling system and start acquisition on enabled interfaces. It is possible to
        enable/disable interfaces both before and after the polling has been started. However, no data will be
        transferred until the polling is started.
        
        `int start_polling(uint32_t dgi_hndl)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :raises: :exc:`DeviceReturnError`
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl

        res = self.dgilib.start_polling(dgi_hndl)
        if self.verbose:
            print(f"\t{res} start_polling")
        if res:
            raise DeviceReturnError(f"start_polling returned: {res}")

    def stop_polling(self, dgi_hndl=None):
        """`stop_polling`
        This function will stop the polling system and stop acquisition on all interfaces.
        
        `int stop_polling(uint32_t dgi_hndl)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :raises: :exc:`DeviceReturnError`
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl

        res = self.dgilib.stop_polling(dgi_hndl)
        if self.verbose:
            print(f"\t{res} stop_polling")
        if res:
            raise DeviceReturnError(f"stop_polling returned: {res}")

    def target_reset(self, dgi_hndl=None, hold_reset=False):
        """`target_reset`
        This function is used to control the state of the reset line connected to the target, if available.
        
        `int target_reset(uint32_t dgi_hndl, bool hold_reset)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *hold_reset* | True will assert reset, false will release it |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :param hold_reset: True will assert reset, false will release it (defaults to False)
        :type hold_reset: bool
        :raises: :exc:`DeviceReturnError`
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl

        res = self.dgilib.target_reset(dgi_hndl, hold_reset)
        if self.verbose:
            print(f"\t{res} target_reset")
        if res:
            raise DeviceReturnError(f"target_reset returned: {res}")

    def interface_list(self, dgi_hndl=None):
        """`interface_list`
        Queries the connected DGI device for available interfaces. Refer to the DGI documentation to resolve the
        ID.
        
        `int interface_list(uint32_t dgi_hndl, unsigned char* interfaces, unsigned char* count)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *interfaces* | Buffer to hold the ID of the available interfaces. Should be able to hold minimum 10 elements, but a larger count should be used to be future proof. |
        | *count* | Pointer to a variable that will be set to the number of interfaces registered in buffer. |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :return: List of available interfaces
        :rtype: list(int)
        :raises: :exc:`DeviceReturnError`
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl

        interfaces = (c_ubyte * NUM_INTERFACES)()
        interfaceCount = c_ubyte()
        res = self.dgilib.interface_list(
            dgi_hndl, byref(interfaces), byref(interfaceCount)
        )
        if self.verbose:
            print(
                f"\t{res} interface_list: {interfaces[:interfaceCount.value]}, interfaceCount: {interfaceCount.value}"
            )
        if res:
            raise DeviceReturnError(f"interface_list returned: {res}")

        return interfaces[: interfaceCount.value], interfaceCount.value

    def interface_enable(self, dgi_hndl=None, interface_id=0, timestamp=True):
        """`interface_enable`
        Enables the specified interface. Note that no data acquisition will begin until a session has been started.

        `int interface_enable(uint32_t dgi_hndl, int interface_id, bool timestamp)`
     
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *interface_id* | The ID of the interface to enable. |
        | *timestamp* | Setting this to true will make the interface use timestamping. Consult the DGI documentation for details on the timestamping option. |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :param interface_id: The ID of the interface to enable (defaults to 0)
        :type interface_id: int
        :param timestamp: Setting this to true will make the interface use timestamping (defaults to True)
        :type timestamp: bool
        :raises: :exc:`DeviceReturnError`
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl

        res = self.dgilib.interface_enable(dgi_hndl, interface_id, timestamp)
        if self.verbose:
            print(f"\t{res} interface_enable: {interface_id}, timestamp: {timestamp}")
        if res:
            raise DeviceReturnError(f"interface_enable returned: {res}")

    def interface_disable(self, dgi_hndl=None, interface_id=0):
        """`interface_disable`
        Disables the specified interface.

        `int interface_disable(uint32_t dgi_hndl, int interface_id)`
     
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *interface_id* | The ID of the interface to disable. |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :param interface_id: The ID of the interface to disable (defaults to 0)
        :type interface_id: int
        :raises: :exc:`DeviceReturnError`
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl

        res = self.dgilib.interface_disable(dgi_hndl, interface_id)
        if self.verbose:
            print(f"\t{res} interface_disable: {interface_id}")
        if res:
            raise DeviceReturnError(f"interface_disable returned: {res}")

    def interface_get_configuration(self, dgi_hndl=None, interface_id=0):
        """`interface_get_configuration`
        Gets the configuration associated with the specified interface. Consult the DGI documentation for details.

        `int interface_get_configuration(uint32_t dgi_hndl, int interface_id, unsigned int* config_id, unsigned int*
        config_value, unsigned int* config_cnt)`
     
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle to connection |
        | *interface_id* | The ID of the interface |
        | *config_id* | Buffer that will hold the ID field for the configuration item |
        | *config_value* | Buffer that will hold the value field for the configuration item |
        | *config_cnt* | Pointer to variable that will hold the count of stored configuration items |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :param interface_id: The ID of the interface (defaults to 0)
        :type interface_id: int
        :return: Tuple of a list of configuration IDs and a list of configuration values
        :rtype: tuple(list(int), list(int))
        :raises: :exc:`DeviceReturnError`
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl
            
        config_id = (c_uint * NUM_CONFIG_IDS)()
        config_value = (c_uint * NUM_CONFIG_IDS)()
        config_cnt = c_uint()
        res = self..interface_get_configuration(dgi_hndl, interface_id, byref(config_id), byref(config_value), byref(config_cnt))
        if self.verbose:
            print(f"\t{res} interface_get_configuration: {interface_id}, config_cnt: {config_cnt.value}")
            for i in range(config_cnt.value):
                print(f"\t\tconfig_id: {config_id[i]},\tvalue: {config_value[i]}")
        if res:
            raise DeviceReturnError(f"interface_get_configuration returned: {res}")
            
        return config_id[:config_cnt], config_value[:config_cnt]

    def interface_set_configuration(self, dgi_hndl=None, interface_id=0, config_id=None, config_value=None, config_cnt=None):
        """`interface_set_configuration`
        Sets the given configuration fields for the specified interface. Consult the DGI documentation for details.

        `int interface_set_configuration(uint32_t dgi_hndl, int interface_id, unsigned int* config_id, unsigned int*
        config_value, unsigned int config_cnt)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle to connection |
        | *interface_id* | The ID of the interface |
        | *config_id* | Buffer that holds the ID field for the configuration items to set |
        | *config_value* | Buffer that holds the value field for the configuration items to set |
        | *config_cnt* | Number of items to set |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection (defaults to self.dgi_hndl)
        :type dgi_hndl: c_uint() or None
        :param interface_id: List that will hold the ID field for the configuration item (defaults to 0)
        :rtype: tuple(list(int), list(int))
        :raises: :exc:`DeviceReturnError`
        """

        if dgi_hndl is None:
            dgi_hndl = self.dgi_hndl
        
        config_id = (c_uint * NUM_CONFIG_IDS)(*config_id)
        config_value = (c_uint * NUM_CONFIG_IDS)(*config_value)
        config_cnt = c_uint(config_cnt)
        res = self.dgilib.interface_set_configuration(dgi_hndl, interface_id, byref(config_id), byref(config_value), config_cnt)
        if self.verbose:
            print(f"\t{res} interface_set_configuration: {interface_id}, config_cnt: {config_cnt.value}")
            for i in range(config_cnt.value):
                print(f"\t\tconfig_id: {config_id[i]},\tvalue: {config_value[i]}")
        if res:
            raise DeviceReturnError(f"interface_get_configuration returned: {res}")

In [21]:
with DGILib(dgilib_path="dgilib") as dgilib:
    dgilib.info()
    dgilib.get_device_name()
    dgilib.get_fw_version()
    sleep(1)

device_count: 1
	0 get_device_serial: b'ATML3138061800001604'
	17 connect


DeviceReturnError: connect returned: 17


        +------------+------------+
        | Parameter  | Description |
        +============+============+
        |  |
        +------------+------------+

In [14]:
c_status = dgilib.connection_status(dgi_hndl)
print("connection_status: {0}".format(c_status))

connection_status: 0


#### `target_reset`
This function is used to control the state of the reset line connected to the target, if available.

A non-zero return value indicates an error.

`int target_reset(uint32_t dgi_hndl, bool hold_reset)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection
*hold_reset* | True will assert reset, false will release it

In [15]:
print("Resetting {0}".format(repr(name.value)))
print(dgilib.target_reset(dgi_hndl, True))
time.sleep(1)
print(dgilib.target_reset(dgi_hndl, False))
print("Done");

Resetting b'SAM L11 Xplained Pro'
0
0
Done


In [16]:
c_status = dgilib.connection_status(dgi_hndl)
print("connection_status: {0}".format(c_status))

connection_status: 0


#### `get_fw_version`
Gets the firmware version of the DGI device connected. Note that this is the version of the DGI device, and not the tool.

A non-zero return value indicates an error.

`int get_fw_version(uint32_t dgi_hndl, unsigned char* major, unsigned char* minor)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection
*major* | Pointer to a variable where the major version will be stored
*minor* | Pointer to a variable where the minor version will be stored

In [17]:
major = c_ubyte()
minor = c_ubyte()
res = dgilib.get_fw_version(dgi_hndl, byref(major), byref(minor))
print("{0} fw_version major: {1}, minor: {2}".format(res, repr(major.value), repr(minor.value)))

0 fw_version major: 3, minor: 2


## Interface

#### `interface_list`
Query the connected DGI device for available interfaces. Refer to the DGI documentation to resolve the ID.

`int interface_list(uint32_t dgi_hndl, unsigned char* interfaces, unsigned char* count)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interfaces* | Buffer to hold the ID of the available interfaces. Should be able to hold minimum 10 elements, but a larger count should be used to be future proof.
*count* | Pointer to a variable that will be set to the number of interfaces registered in buffer.

In [18]:
interfaces = create_string_buffer(NUM_INTERFACES)
interfaceCount = c_ubyte()

In [19]:
res = dgilib.interface_list(dgi_hndl, byref(interfaces), byref(interfaceCount))
print("{0} interface_list: {1}, interfaceCount: {2}".format(res, interfaces[:], repr(interfaceCount.value)))

0 interface_list: b'\x00 "0@AP\x00\x00\x00', interfaceCount: 7


#### `interface_get_configuration`
Gets the configuration associated with the specified interface. Consult the DGI documentation for details.

A non-zero return value indicates an error.

`int interface_get_configuration(uint32_t dgi_hndl, int interface_id, unsigned int* config_id, unsigned int* config_value, unsigned int* config_cnt)`

Parameter   | Description
-------------  | -------------
| *dgi_hndl* | Handle to connection |
| *interface_id* | The ID of the interface |
| *config_id* | Buffer that will hold the ID field for the configuration item |
| *config_value* | Buffer that will hold the value field for the configuration item |
| *config_cnt* | Pointer to variable that will hold the count of stored configuration items |

In [20]:
config_id = (c_uint * NUM_CONFIG_IDS)()
config_value = (c_uint * NUM_CONFIG_IDS)()
config_cnt = c_uint()
res = dgilib.interface_get_configuration(dgi_hndl, INTERFACE_TIMESTAMP, byref(config_id), byref(config_value), byref(config_cnt))
print("{0} interface_get_configuration: {1}, config_cnt: {2}".format(res, INTERFACE_TIMESTAMP, repr(config_cnt.value)))
for i in range(config_cnt.value):
    print("\tconfig_id: {0},\tvalue: {1}".format(config_id[i], config_value[i]))

0 interface_get_configuration: 0, config_cnt: 2
	config_id: 0,	value: 32
	config_id: 1,	value: 60000000


In [21]:
timerPrescaler = config_value[0]
timerFrequency = config_value[1]
timerFactor = timerPrescaler / timerFrequency
print("timerFactor: {0}".format(timerFactor))

timerFactor: 5.333333333333333e-07


#### `interface_enable`
Enables the specified interface. Note that no data acquisition will begin until a session has been started.

A non-zero return value indicates an error.

`int interface_enable(uint32_t dgi_hndl, int interface_id, bool timestamp)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface to enable
*timestamp* | Setting this to true will make the interface use timestamping. Consult the DGI documentation for details on the timestamping option.

In [22]:
interface_id = INTERFACE_GPIO;
enableTimestamp = True;
res = dgilib.interface_enable(dgi_hndl, interface_id, enableTimestamp);
print("{0} interface_enable: {1}, enableTimestamp: {2}\n".format(res, interface_id, enableTimestamp))

0 interface_enable: 48, enableTimestamp: True



In [23]:
for i in range(NUM_CONFIG_IDS):
    config_id[i] = 0
    config_value[i] = 0
res = dgilib.interface_get_configuration(dgi_hndl, interface_id, byref(config_id), byref(config_value), byref(config_cnt))
print("{0} interface_get_configuration: {1}, config_cnt: {2}".format(res, interface_id, repr(config_cnt.value)))
for i in range(config_cnt.value):
    print("\tconfig_id: {0},\tvalue: {1}".format(config_id[i], config_value[i]))

0 interface_get_configuration: 48, config_cnt: 2
	config_id: 0,	value: 0
	config_id: 1,	value: 0


#### `interface_set_configuration`
Sets the given configuration fields for the specified interface. Consult the DGI documentation for details.

A non-zero return value indicates an error.

`int interface_set_configuration(uint32_t dgi_hndl, int interface_id, unsigned int* config_id, unsigned int*
config_value, unsigned int config_cnt)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface
*config_id* | Buffer that holds the ID field for the configuration items to set
*config_value* | Buffer that holds the value field for the configuration items to set
*config_cnt* | Number of items to set

In [24]:
config_value[0] = 0b1111
config_value[1] = 0b0000
res = dgilib.interface_set_configuration(dgi_hndl, interface_id, byref(config_id), byref(config_value), config_cnt)
print("{0} interface_set_configuration: {1}, config_cnt: {2}".format(res, interface_id, repr(config_cnt.value)))
for i in range(config_cnt.value):
    print("\tconfig_id: {0},\tvalue: {1}".format(config_id[i], config_value[i]))

0 interface_set_configuration: 48, config_cnt: 2
	config_id: 0,	value: 15
	config_id: 1,	value: 0


#### `interface_clear_buffer`
Clears the data in the buffers for the specified interface.

A non-zero return value indicates an error.

`int interface_clear_buffer(uint32_t dgi_hndl, int interface_id)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface

In [25]:
res = dgilib.interface_clear_buffer(dgi_hndl, interface_id)
print("{0} interface_clear_buffer: {1}".format(res, interface_id))

0 interface_clear_buffer: 48


#### `start_polling`
This function will start the polling system and start acquisition on enabled interfaces. It is possible to enable/disable interfaces both before and after the polling has been started. However, no data will be transferred until the polling is started.

A non-zero return value indicates an error.

`int start_polling(uint32_t dgi_hndl)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection

In [26]:
res = dgilib.start_polling(dgi_hndl)
print("{0} start_polling".format(res))

0 start_polling


#### `interface_read_data`
Reads the data received on the specified interface. This should be called regularly to avoid overflows in the system. DGILib can buffer 10M samples.

A non-zero return value indicates an error.

`int interface_read_data(uint32_t dgi_hndl, int interface_id, unsigned char* buffer, unsigned long long* timestamp, int* length, unsigned int* ovf_index, unsigned int* ovf_length, unsigned int* ovf_entry_count)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface
*buffer* | Buffer that will hold the received data. The buffer must have allocated 10M elements.
*timestamp* | If timestamp is enabled for the interface, the buffer that will hold the received data. The buffer must have allocated 10M elements. Otherwise send 0.
*length* | Pointer to a variable that will hold the count of elements received
*ovf_index* | Reserved. Set to 0.
*ovf_length* | Reserved. Set to 0.
*ovf_entry_count* | Reserved. Set to 0. Could be set to a pointer to a variable that can be used as an indicator of overflows. Overflow would be indicated by non-zero value.

In [27]:
# readBuffer = create_string_buffer(BUFFER_SIZE)
readBuffer = (c_ubyte * BUFFER_SIZE)()
timestamp = (c_ulonglong * BUFFER_SIZE)()
length = c_uint(0)
ovf_index = c_uint(0)
ovf_length = c_uint(0)
ovf_entry_count = c_uint(0)
res = dgilib.interface_read_data(dgi_hndl, interface_id, readBuffer, timestamp, byref(length), byref(ovf_index), byref(ovf_length), byref(ovf_entry_count));
print("{0} interface_read_data: {1}, length: {2}".format(res, interface_id, length.value))
for i in range(length.value):
    print("\t{0}: buffer: {1}, timestamp: {2}, time: {3}".format(i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor))

0 interface_read_data: 48, length: 3
	0: buffer: 12, timestamp: 3918, time: 0.0020896
	1: buffer: 13, timestamp: 89722, time: 0.047851733333333334
	2: buffer: 15, timestamp: 108739, time: 0.057994133333333336


#### `interface_write_data`
Writes data to the specified interface. A maximum of 255 elements can be written each time. An error return code will be given if data hasn't been written yet.

A non-zero return value indicates an error. An error will be returned if the interface is still in the process of writing data. Wait a while and try again. The function get_connection_status can be used to verify if there is an error condition.

`int interface_write_data(uint32_t dgi_hndl, int interface_id, unsigned char* buffer, int* length)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface
*buffer* | Buffer that will hold the received data. The buffer must have allocated 10M elements.
*length* | Pointer to a variable that will hold the count of elements received

In [28]:
# writeBuffer = (c_ubyte * NUM_CONFIG_IDS)()
# writeLength = c_int(2)
# res = dgilib.interface_write_data(dgi_hndl, interface_id, byref(writeBuffer), byref(writeLength))
# print("{0} interface_write_data: {1}, length: {2}".format(res, interface_id, writeLength.value))

In [29]:
# res = dgilib.interface_read_data(dgi_hndl, interface_id, readBuffer, timestamp, byref(length), byref(ovf_index), byref(ovf_length), byref(ovf_entry_count));
# print("{0} interface_read_data: {1}, length: {2}".format(res, interface_id, length.value))
# for i in range(length.value):
#     print("\t{0}: buffer: {1}, timestamp: {2}, time: {3}".format(i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor))

## Auxiliary

### Power
The power interface (as found on some EDBG kits and Power Debugger) uses a protocol stream and calibration scheme that can be tricky to get right. The data rates are also relatively high and the calibration procedure could cause issues if not handled efficiently. Therefore some auxiliary functions to help with this have been made to perform parsing and calibration.

#### `auxiliary_power_initialize`
Initializes the power parser.

A non-zero return value indicates an error.

`int auxiliary_power_initialize(uint32_t* power_hndl_p, uint32_t dgi_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl_p* | Pointer to variable that will hold the handle to the power parser
*dgi_hndl* | Handle to connection

In [30]:
power_hndl = c_uint()
res = dgilib.auxiliary_power_initialize(byref(power_hndl), dgi_hndl)
print("{0} auxiliary_power_initialize".format(res))

0 auxiliary_power_initialize


#### `auxiliary_power_get_circuit_type`
Gets the type of power circuit.

A non-zero return value indicates an error.

`int auxiliary_power_get_circuit_type(uint32_t power_hndl, int* circuit)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*circuit* | Pointer to a variable that will hold the circuit type: OLD_XAM = 0x00, XAM = 0x10, PAM = 0x11, UNKNOWN = 0xFF

In [31]:
circuit = c_int()
res = dgilib.auxiliary_power_get_circuit_type(power_hndl, byref(circuit))
print("{0} auxiliary_power_get_circuit_type: {1}".format(res, circuit.value))

0 auxiliary_power_get_circuit_type: 16


#### `auxiliary_power_calibration_is_valid`
Checks the status of the stored calibration.

Returns true if the calibration is valid, false otherwise. Unity gain and offset will be used.

`bool auxiliary_power_calibration_is_valid(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [32]:
res = dgilib.auxiliary_power_calibration_is_valid(power_hndl)
print("auxiliary_power_calibration_is_valid: {0}".format(res))

auxiliary_power_calibration_is_valid: 1


#### `auxiliary_power_calibration_is_valid`
Checks the status of the stored calibration.

Returns true if the calibration is valid, false otherwise. Unity gain and offset will be used.

`bool auxiliary_power_calibration_is_valid(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*type* | Type of calibration to trigger. See the DGI documentation for details.

In [33]:
calibrationType = circuit
res = dgilib.auxiliary_power_trigger_calibration(power_hndl, calibrationType)
print("{0} auxiliary_power_trigger_calibration".format(res))
if (not res):
    print("Calibrating {0}".format(name.value))
    while (dgilib.auxiliary_power_get_status(power_hndl) == 0x03):
        print("Calibrating...")
        time.sleep(0.1)
    print("Done")

0 auxiliary_power_trigger_calibration
Calibrating b'SAM L11 Xplained Pro'
Calibrating...
Calibrating...
Done


#### `auxiliary_power_get_status`
Gets the status of the power parser.

Return codes
- `IDLE` = 0x00
- `RUNNING` = 0x01
- `DONE` = 0x02
- `CALIBRATING` = 0x03
- `INIT_FAILED` = 0x10
- `OVERFLOWED` = 0x11
- `USB_DISCONNECTED` = 0x12
- `CALIBRATION_FAILED` = 0x20

`int auxiliary_power_get_status(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [34]:
powerStatus = c_int(dgilib.auxiliary_power_get_status(power_hndl))
print("power_status: {0}".format(powerStatus.value))

power_status: 0


#### `auxiliary_power_get_calibration`
Gets the raw calibration read from the tool.

A non-zero return value indicates an error.

`int auxiliary_power_get_calibration(uint32_t power_hndl, uint8_t* data, size_t length)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*data* | Buffer that will hold the read raw calibration data
*length* | Number of raw calibration bytes to fetch. See the DGI documentation for number of bytes.

In [35]:
calibrationData = (c_ubyte * NUM_CALIBRATION)()
calibrationLength = c_size_t(NUM_CALIBRATION)
res = dgilib.auxiliary_power_get_calibration(power_hndl, byref(calibrationData), calibrationLength)
print("{0} auxiliary_power_get_calibration".format(res))
# for i in range(calibrationLength.value):
#     print("\t{0}: {1}".format(i, calibrationData[i]))

208 auxiliary_power_get_calibration


#### `auxiliary_power_register_buffer_pointers`
Registers a set of pointers to be used for storing the calibrated power data. The buffers can then be locked by auxiliary_power_lock_data_for_reading, and the data directly read from the specified buffers. Zero-pointers can be specified to get the buffers allocated within DGILib. This requires the data to be fetched using auxiliary_power_copy_data.

A non-zero return value indicates an error.

`int auxiliary_power_register_buffer_pointers(uint32_t power_hndl, float* buffer, double* timestamp, size_t* count, size_t max_count, int channel, int type)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*buffer* | Buffer that will hold the samples. Set to 0 for automatically allocated.
*timestamp* | Buffer that will hold the timestamp for the samples. Set to 0 for automatically allocated.
*count* | Pointer to a variable that will hold the count of samples. Set to 0 for automatically allocated.
*max_count* | Number of samples that can fit into the specified buffers. Or size of automatically allocated buffers.
*channel* | Power channel for this buffer: A = 0, B = 1 (Power Debugger specific)
*type* | Type of power data: Current = 0, Voltage = 1, Range = 2

In [36]:
powerBuffer = (c_float * BUFFER_SIZE)()
powerTimestamp = (c_double * BUFFER_SIZE)()
powerCount = c_size_t()
max_count = (c_size_t * BUFFER_SIZE)()
channel = c_int(0)
powerType = c_int(0)
res = dgilib.auxiliary_power_register_buffer_pointers(power_hndl, byref(powerBuffer), byref(powerTimestamp), byref(powerCount), max_count, channel, powerType);
# res = dgilib.auxiliary_power_register_buffer_pointers(power_hndl, 0, 0, 0, max_count, channel, powerType);
print("{0} auxiliary_power_register_buffer_pointers".format(res))

0 auxiliary_power_register_buffer_pointers


In [37]:
powerStatus = c_int(dgilib.auxiliary_power_get_status(power_hndl))
print("power_status: {0}".format(powerStatus.value))

power_status: 0


#### `auxiliary_power_start`
Starts parsing of power data. The power and power sync interfaces are enabled automatically, but note that it is necessary to start the polling separately. This only starts the parser that consumes data from the DGILib buffer.

A non-zero return value indicates an error.

`int auxiliary_power_start(uint32_t power_hndl, int mode, int parameter)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*mode* | Sets the mode of capture.
 | 0 - continuous capturing which requires the user to periodically consume the data.
 | 1 - oneshot capturing that captures data until the buffer has been read once, has been filled or the time from the first received sample in seconds equals the specified parameter.
*parameter* | Mode specific

In [38]:
mode = c_int(0)
parameter = c_int(0)
res = dgilib.auxiliary_power_start(power_hndl, mode, parameter);
print("{0} auxiliary_power_start".format(res))

0 auxiliary_power_start


In [39]:
powerStatus = c_int(dgilib.auxiliary_power_get_status(power_hndl))
print("power_status: {0}".format(powerStatus.value))

power_status: 0


In [40]:
#     int loggedSamples = 0;
#     printf("Waiting to log some data \n");
#     for (int i = 0; i < 25; i++) {
#         auxiliary_power_lock_data_for_reading(power_hndl);
#         auxiliary_power_copy_data(power_hndl, powerBuffer+loggedSamples, powerTimestamp+loggedSamples, &powerCount, max_count, channel, powerType);
#         auxiliary_power_free_data(power_hndl);
#         loggedSamples += powerCount;
#         printf(".%d", powerCount);
# //        printf(".");
#         for (int j = 0;j < WAIT_ITERATIONS; j+=10) {}
#     }
#     printf(" Done\n");

In [41]:
powerStatus = c_int(dgilib.auxiliary_power_get_status(power_hndl))
print("power_status: {0}".format(powerStatus.value))

power_status: 1


In [42]:
#     interface_id = INTERFACE_GPIO;
#     res = interface_read_data(dgi_hndl, interface_id, readBuffer, timestamp, &length, &ovf_index, &ovf_length, &ovf_entry_count);
#     printf("%d interface_read_data: 0x%.2x, length: %d\n", res, interface_id, length);
#     for (int i = 0; i < length; ++i) {
#         printf("\t%.6d: buffer: %.4u, timestamp: %.4llu, time: %lf\n", i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor);
#     }

#### `auxiliary_power_lock_data_for_reading`
Blocks the parsing thread from accessing all the buffers. This must be called before the user application code accesses the buffers, or a call to auxiliary_power_copy_data is made. Afterwards auxiliary_power_free_data must be called. Minimize the amount of time between locking and freeing to avoid buffer overflows.

A non-zero return value indicates an error.

`int auxiliary_power_lock_data_for_reading(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [43]:
res = dgilib.auxiliary_power_lock_data_for_reading(power_hndl)
print("{0} auxiliary_power_lock_data_for_reading".format(res))

0 auxiliary_power_lock_data_for_reading


#### `auxiliary_power_copy_data`
Copies parsed power data into the specified buffer. Remember to lock the buffers first. If the count parameter is the same as max_count there is probably more data to be read. Do another read to get the remaining data.

A non-zero return value indicates an error.

`int auxiliary_power_copy_data(uint32_t power_hndl, float* buffer, double* timestamp, size_t* count, size_t
max_count, int channel, int type)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*buffer* | Buffer that will hold the data
*timestamp* | Buffer that will hold the timestamps
*count* | Pointer to a variable that will hold the count of elements copied
*max_count* | Maximum number of elements that the buffer can hold
*channel* | Power channel for this buffer: A = 0, B = 1 (Power Debugger specific)
*type* | Type of power data: Current = 0, Voltage = 1, Range = 2

In [44]:
res = dgilib.auxiliary_power_copy_data(power_hndl, powerBuffer, powerTimestamp, byref(powerCount), max_count, channel, powerType);
print("{0} auxiliary_power_copy_data: {1} samples".format(res, powerCount.value))

0 auxiliary_power_copy_data: 0 samples


#### `auxiliary_power_free_data`
Clears the power data buffers and allows the power parser to continue.

A non-zero return value indicates an error.

`int auxiliary_power_free_data(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [45]:
res = dgilib.auxiliary_power_free_data(power_hndl)
print("{0} auxiliary_power_free_data".format(res))

0 auxiliary_power_free_data


#### `auxiliary_power_stop`
Stops parsing of power data.

A non-zero return value indicates an error.

`int auxiliary_power_stop(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [46]:
res = dgilib.auxiliary_power_stop(power_hndl)
print("{0} auxiliary_power_stop".format(res))

0 auxiliary_power_stop


In [47]:
print("captured power data:")
for i in range(powerCount.value):
    print("\t{0}: buffer: {1}, timestamp: {2}".format(i, powerBuffer[i], powerTimestamp[i]))

captured power data:
	0: buffer: 0.0009178051841445267, timestamp: 0.7102284048
	1: buffer: 0.00029516639187932014, timestamp: 0.7102944
	2: buffer: 0.00033206219086423516, timestamp: 0.7103603951999999
	3: buffer: 0.0003566594095900655, timestamp: 0.7104263904
	4: buffer: 0.0003443608002271503, timestamp: 0.7104923855999999
	5: buffer: 0.0003443608002271503, timestamp: 0.7105583808
	6: buffer: 0.0003443608002271503, timestamp: 0.710624376
	7: buffer: 0.0003443608002271503, timestamp: 0.7106903712
	8: buffer: 0.0003443608002271503, timestamp: 0.7107563664
	9: buffer: 0.00033206219086423516, timestamp: 0.7108223616
	10: buffer: 0.0003443608002271503, timestamp: 0.7108883568
	11: buffer: 0.00033206219086423516, timestamp: 0.710954352
	12: buffer: 0.0003443608002271503, timestamp: 0.7110203472
	13: buffer: 0.00033206219086423516, timestamp: 0.7110863424
	14: buffer: 0.0003443608002271503, timestamp: 0.7111523376
	15: buffer: 0.00031976361060515046, timestamp: 0.7112183328
	16: buffer: 0.0

In [48]:
powerStatus = c_int(dgilib.auxiliary_power_get_status(power_hndl))
print("power_status: {0}".format(powerStatus.value))

power_status: 0


In [49]:
#     interface_id = INTERFACE_GPIO;
#     res = interface_read_data(dgi_hndl, interface_id, readBuffer, timestamp, &length, &ovf_index, &ovf_length, &ovf_entry_count);
#     printf("%d interface_read_data: 0x%.2x, length: %d\n", res, interface_id, length);
#     for (int i = 0; i < length; ++i) {
#         printf("\t%.6d: buffer: %.4u, timestamp: %.4llu, time: %lf\n", i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor);
#     }

In [50]:
#     res = interface_read_data(dgi_hndl, INTERFACE_POWER_SYNC, readBuffer, timestamp, &length, &ovf_index, &ovf_length, &ovf_entry_count);
#     printf("%d interface_read_data: 0x%.2x, length: %d\n", res, INTERFACE_POWER_SYNC, length);
#     for (int i = 0; i < length; ++i) {
#         printf("\t%.6d: buffer: %.4u, timestamp: %.4llu, time: %lf\n", i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor);
#     }

## Uninitialize

Here are all the functions to shutdown the device gracefully and avoid the dreaded mode 17.

#### `auxiliary_power_unregister_buffer_pointers`
Unregisters the pointers for the specified power channel.

A non-zero return value indicates an error.

`int auxiliary_power_unregister_buffer_pointers(uint32_t power_hndl, int channel, int type)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser
*channel* | Power channel for this buffer: A = 0, B = 1 (Power Debugger specific)
*type* | Type of power data: Current = 0, Voltage = 1, Range = 2

In [51]:
res = dgilib.auxiliary_power_unregister_buffer_pointers(power_hndl, channel, powerType)
print("{0} auxiliary_power_unregister_buffer_pointers".format(res))

0 auxiliary_power_unregister_buffer_pointers


#### `auxiliary_power_uninitialize`
Uninitializes the power parser.

A non-zero return value indicates an error.

`int auxiliary_power_uninitialize(uint32_t power_hndl)`

Parameter   | Description
-------------  | -------------
*power_hndl* | Handle to the power parser

In [52]:
res = dgilib.auxiliary_power_uninitialize(power_hndl)
print("{0} auxiliary_power_uninitialize".format(res))

0 auxiliary_power_uninitialize


#### `stop_polling`
This function will stop the polling system and stop acquisition on all interfaces.

A non-zero return value indicates an error.

`int stop_polling(uint32_t dgi_hndl)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection

In [53]:
res = dgilib.stop_polling(dgi_hndl)
print("{0} stop_polling".format(res))

0 stop_polling


#### `interface_disable`
Disables the specified interface.

A non-zero return value indicates an error.

`int interface_disable(uint32_t dgi_hndl, int interface_id)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle to connection
*interface_id* | The ID of the interface to enable

In [54]:
interface_id = INTERFACE_GPIO
res = dgilib.interface_disable(dgi_hndl, interface_id)
print("{0} interface_disable: {1}".format(res, interface_id))

#     res = interface_disable(dgi_hndl, interface_id);
#     printf("%d interface_disable: 0x%.2x\n", res, interface_id);

#     res = interface_disable(dgi_hndl, INTERFACE_POWER_SYNC);
#     printf("%d interface_disable: 0x%.2x\n", res, INTERFACE_POWER_SYNC);

#     res = interface_disable(dgi_hndl, INTERFACE_TIMESTAMP);
#     printf("%d interface_disable: 0x%.2x\n", res, INTERFACE_TIMESTAMP);

0 interface_disable: 48


#### `disconnect`
Closes the specified connection.

A non-zero return value indicates an error.

`int disconnect(uint32_t dgi_hndl)`

Parameter   | Description
-------------  | -------------
*dgi_hndl* | Handle of the connection

In [55]:
res = dgilib.disconnect(dgi_hndl)
print("{0} disconnect".format(res))

0 disconnect
